<a href="https://colab.research.google.com/github/jsjj10002/FackVoiceClassfication/blob/main/%EC%9D%8C%EC%84%B1_%ED%8C%90%EB%B3%84_%EB%AA%A8%EB%8D%B8_CNN%2B%EC%96%91%EB%B0%A9%ED%96%A5LSTM%2Bk_fold_%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사용 데이터셋

[The Fake-or-Real (FoR) Dataset](https://www.kaggle.com/datasets/mohammedabdeldayem/the-fake-or-real-dataset/data)

The Fake-or-Real (FoR) dataset is a collection of more than 195,000 utterances from real humans and computer generated speech. The dataset can be used to train classifiers to detect synthetic speech. The dataset is published in four versions: for-original, for-norm, for-2sec and for-rerec.

The first version, named for-original, contains the files as collected from the speech sources, without any modification (balanced version).

***The second version, called for-norm, contains the same files, but balanced in terms of gender and class and normalized in terms of sample rate, volume and number of channels.***

The third one, named for-2sec is based on the second one, but with the files truncated at 2 seconds.

The last version, named for-rerec, is a rerecorded version of the for-2second dataset, to simulate a scenario where an attacker sends an utterance through a voice channel (i.e. a phone call or a voice message).

사용한 버전: 정규화 버젼

# 케글 API 불러온 후 데이터셋 다운로드


In [ ]:
!gdown https://drive.google.com/uc?id=1Mnp2CPxa6hDi7_hkwUrleP6pQqulrXz0

Downloading...
From: https://drive.google.com/uc?id=1Mnp2CPxa6hDi7_hkwUrleP6pQqulrXz0
To: /content/kaggle.json
100% 67.0/67.0 [00:00<00:00, 233kB/s]


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mohammedabdeldayem/the-fake-or-real-dataset

Dataset URL: https://www.kaggle.com/datasets/mohammedabdeldayem/the-fake-or-real-dataset
License(s): GNU Lesser General Public License 3.0
100% 16.0G/16.0G [02:43<00:00, 197MB/s]
100% 16.0G/16.0G [02:43<00:00, 105MB/s]


In [ ]:
%cd /content
!mkdir the-fake-or-real-dataset
!unzip -qq "/content/the-fake-or-real-dataset.zip" -d the-fake-or-real-dataset/

/content


# 1. 음성파일 경로 수집

In [ ]:
import os
import pandas as pd

# 기본 경로 설정
base_path = "/content/the-fake-or-real-dataset/for-norm/for-norm"

# 카테고리와 타입 정의
categories = ['testing', 'training', 'validation']
types = ['fake', 'real']

# 모든 파일 경로 수집
data = []
for category in categories:
    for type_ in types:
        dir_path = os.path.join(base_path, category, type_)
        for filename in os.listdir(dir_path):
            if filename.endswith('.wav'):
                full_path = os.path.join(dir_path, filename)
                data.append({'path': full_path, 'label': type_, 'category': category})

# DataFrame 생성
path_df = pd.DataFrame(data)
print("Dataframe shape:", path_df.shape)
print(path_df.head())


Dataframe shape: (69300, 3)
                                                path label category
0  /content/the-fake-or-real-dataset/for-norm/for...  fake  testing
1  /content/the-fake-or-real-dataset/for-norm/for...  fake  testing
2  /content/the-fake-or-real-dataset/for-norm/for...  fake  testing
3  /content/the-fake-or-real-dataset/for-norm/for...  fake  testing
4  /content/the-fake-or-real-dataset/for-norm/for...  fake  testing


# 2.MFCC 특징 추출

In [ ]:
import librosa
import numpy as np

def extract_mfcc(file_path):
    try:
        audio, sample_rate = librosa.load(file_path, sr=None)
        if audio.size == 0:
            return None
        mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)
        return mfcc_mean
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# MFCC 특징을 병렬로 추출
from multiprocessing import Pool

def process_row(row):
    mfcc = extract_mfcc(row['path'])
    if mfcc is not None:
        return {'path': row['path'], 'mfcc': mfcc, 'label': row['label'], 'category': row['category']}
    else:
        return None

# 데이터프레임에서 병렬 처리를 위해 데이터를 리스트로 변환
data_list = path_df.to_dict('records')
# 병렬 처리 설정
with Pool(processes=4) as pool:
    results = pool.map(process_row, data_list)

# None을 제거하고 데이터프레임 생성
mfcc_data = [result for result in results if result is not None]
mfcc_df = pd.DataFrame(mfcc_data)

print("MFCC DataFrame shape:", mfcc_df.shape)
print(mfcc_df.head())


/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1891
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1837
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1690
  warnings.warn(


MFCC DataFrame shape: (69298, 4)
                                                path  \
0  /content/the-fake-or-real-dataset/for-norm/for...   
1  /content/the-fake-or-real-dataset/for-norm/for...   
2  /content/the-fake-or-real-dataset/for-norm/for...   
3  /content/the-fake-or-real-dataset/for-norm/for...   
4  /content/the-fake-or-real-dataset/for-norm/for...   

                                                mfcc label category  
0  [-121.118385, 104.168976, -17.292633, 7.035876...  fake  testing  
1  [-133.96411, 77.051575, -3.4393408, 11.727731,...  fake  testing  
2  [-153.86955, 123.38464, -12.190086, 23.369476,...  fake  testing  
3  [-126.404106, 96.755585, -7.130523, 5.172734, ...  fake  testing  
4  [-136.06691, 120.67819, -12.658545, 18.204365,...  fake  testing  


# 3.데이터셋 분할

In [ ]:
train_df = mfcc_df[mfcc_df['category'] == 'training']
val_df = mfcc_df[mfcc_df['category'] == 'validation']
test_df = mfcc_df[mfcc_df['category'] == 'testing']

print("Training Data Shape:", train_df.shape)
print("Validation Data Shape:", val_df.shape)
print("Testing Data Shape:", test_df.shape)


Training Data Shape: (53866, 4)
Validation Data Shape: (10798, 4)
Testing Data Shape: (4634, 4)


# 4,데이터 준비

In [ ]:
X_train = np.vstack(train_df['mfcc'].apply(lambda x: np.array(x)).values)
Y_train = train_df['label'].values

X_val = np.vstack(val_df['mfcc'].apply(lambda x: np.array(x)).values)
Y_val = val_df['label'].values

X_test = np.vstack(test_df['mfcc'].apply(lambda x: np.array(x)).values)
Y_test = test_df['label'].values

print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_val shape:", X_val.shape)
print("Y_val shape:", Y_val.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)


X_train shape: (53866, 13)
Y_train shape: (53866,)
X_val shape: (10798, 13)
Y_val shape: (10798,)
X_test shape: (4634, 13)
Y_test shape: (4634,)


## 필요 라이브러리

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# 5.데이터 정규화

In [ ]:
# StandardScaler를 사용하여 특징 데이터의 스케일을 조정합니다.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# 6.레이블 인코딩

In [ ]:
# 문자열 레이블을 숫자로 변환.
encoder = LabelEncoder()
Y_train = encoder.fit_transform(Y_train)
Y_val = encoder.transform(Y_val)
Y_test = encoder.transform(Y_test)

## 6-1 원-핫 인코딩

In [ ]:
# 분류 작업을 위해 레이블을 원-핫 인코딩 형식으로 변환.
Y_train = to_categorical(Y_train)
Y_val = to_categorical(Y_val)
Y_test = to_categorical(Y_test)

# 7.모델 구축

In [ ]:
# 간단한 신경망 모델을 구성. 입력 레이어, 두 개의 숨겨진 레이어, 그리고 출력 레이어로 구성.
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),  # 입력 레이어
    Dense(128, activation='relu'),                                   # 은닉층
    Dense(Y_train.shape[1], activation='softmax')                    # 출력 레이어, 클래스 수만큼 출력 노드를 가짐
])
# 모델 컴파일. 손실 함수로는 'categorical_crossentropy'를 사용하며, 최적화기로는 'adam'을 사용.
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 8.모델 훈련

In [ ]:
# 모델 훈련
# 모델을 훈련 데이터로 훈련하고, 검증 데이터로 각 에포크마다 성능을 평가합니다.
model.fit(X_train, Y_train, epochs=10, validation_data=(X_val, Y_val))

Epoch 1/10
1684/1684 [==============================] - 8s 4ms/step - loss: 0.1617 - accuracy: 0.9398 - val_loss: 0.0961 - val_accuracy: 0.9674
Epoch 2/10
1684/1684 [==============================] - 9s 5ms/step - loss: 0.0784 - accuracy: 0.9726 - val_loss: 0.0742 - val_accuracy: 0.9748
Epoch 3/10
1684/1684 [==============================] - 6s 4ms/step - loss: 0.0620 - accuracy: 0.9782 - val_loss: 0.0641 - val_accuracy: 0.9766
Epoch 4/10
1684/1684 [==============================] - 4s 2ms/step - loss: 0.0538 - accuracy: 0.9811 - val_loss: 0.0771 - val_accuracy: 0.9716
Epoch 5/10
1684/1684 [==============================] - 4s 2ms/step - loss: 0.0476 - accuracy: 0.9831 - val_loss: 0.0535 - val_accuracy: 0.9806
Epoch 6/10
1684/1684 [==============================] - 5s 3ms/step - loss: 0.0426 - accuracy: 0.9850 - val_loss: 0.0578 - val_accuracy: 0.9795
Epoch 7/10
1684/1684 [==============================] - 5s 3ms/step - loss: 0.0392 - accuracy: 0.9861 - val_loss: 0.0603 - val_accuracy:

# 9.모델평가

In [ ]:
# 훈련된 모델을 테스트 데이터로 평가.
test_loss, test_accuracy = model.evaluate(X_test, Y_test)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")

145/145 [==============================] - 0s 2ms/step - loss: 1.9276 - accuracy: 0.5050
Test Accuracy: 50.50%


# 10. CNN + LSTM

In [31]:
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, LSTM, Flatten

# 모델 구성
model = Sequential([
    # 첫 번째 합성곱 레이어
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.25),

    # 두 번째 합성곱 레이어
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.25),

    # LSTM 레이어
    LSTM(128, return_sequences=True),
    Dropout(0.25),

    # 완전 연결 레이어
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    # 출력 레이어
    Dense(Y_train.shape[1], activation='softmax')
])
# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# 모델 요약
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 11, 64)            256       
                                                                 
 max_pooling1d_6 (MaxPoolin  (None, 5, 64)             0         
 g1D)                                                            
                                                                 
 dropout_14 (Dropout)        (None, 5, 64)             0         
                                                                 
 conv1d_7 (Conv1D)           (None, 3, 128)            24704     
                                                                 
 max_pooling1d_7 (MaxPoolin  (None, 1, 128)            0         
 g1D)                                                            
                                                                 
 dropout_15 (Dropout)        (None, 1, 128)           

## 10-1 모델훈련

In [34]:
model.fit(X_train, Y_train, epochs=20, validation_data=(X_val, Y_val), batch_size=32)

Epoch 1/20
1684/1684 [==============================] - 21s 10ms/step - loss: 0.3818 - accuracy: 0.8272 - val_loss: 0.2466 - val_accuracy: 0.8982
Epoch 2/20
1684/1684 [==============================] - 16s 9ms/step - loss: 0.2768 - accuracy: 0.8835 - val_loss: 0.2172 - val_accuracy: 0.9129
Epoch 3/20
1684/1684 [==============================] - 16s 9ms/step - loss: 0.2484 - accuracy: 0.8967 - val_loss: 0.2009 - val_accuracy: 0.9184
Epoch 4/20
1684/1684 [==============================] - 15s 9ms/step - loss: 0.2308 - accuracy: 0.9050 - val_loss: 0.1812 - val_accuracy: 0.9261
Epoch 5/20
1684/1684 [==============================] - 16s 9ms/step - loss: 0.2193 - accuracy: 0.9106 - val_loss: 0.1789 - val_accuracy: 0.9263
Epoch 6/20
1684/1684 [==============================] - 17s 10ms/step - loss: 0.2131 - accuracy: 0.9130 - val_loss: 0.1626 - val_accuracy: 0.9347
Epoch 7/20
1684/1684 [==============================] - 17s 10ms/step - loss: 0.2026 - accuracy: 0.9170 - val_loss: 0.1596 - val

## 10-2.모델평가

In [35]:
test_loss, test_accuracy = model.evaluate(X_test, Y_test)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")

145/145 [==============================] - 1s 5ms/step - loss: 0.9543 - accuracy: 0.5863
Test Accuracy: 58.63%


# 11. CNN, 양방향 LSTM, L1,L2정규화,k-fold교차검증을 적용

In [42]:
from sklearn.model_selection import KFold
from tensorflow.keras.layers import Dropout, Conv1D, MaxPooling1D, Bidirectional, LSTM, Flatten
from tensorflow.keras.regularizers import l1_l2

## 11-1. 모델 구축

In [36]:
def build_advanced_model(input_shape, output_units):
    model = Sequential([
        # 첫 번째 합성곱 레이어와 풀링
        Conv1D(64, kernel_size=5, activation='relu', padding='same', input_shape=input_shape, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        # 두 번째 합성곱 레이어와 풀링
        Conv1D(128, kernel_size=5, activation='relu', padding='same', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        # 세 번째 합성곱 레이어와 풀링
        Conv1D(256, kernel_size=5, activation='relu', padding='same', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        # 양방향 LSTM 레이어
        Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))),
        Dropout(0.3),

        # 데이터 평탄화
        Flatten(),

        # 완전 연결 레이어
        Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
        Dropout(0.5),

        # 출력 레이어
        Dense(output_units, activation='softmax')
    ])

    # 컴파일
    model.compile(optimizer=Adam(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


## 11-2 k-fold 교차검즘 실행

In [38]:
# X와 Y 데이터 병합
X = np.concatenate([X_train, X_val, X_test], axis=0)
Y = np.concatenate([Y_train, Y_val, Y_test], axis=0)
print("X shape:", X.shape)
print("Y shape:", Y.shape)

X shape: (69298, 13)
Y shape: (69298, 2)


In [43]:

# k-Fold 교차 검증 설정
n_splits = 5
input_shape = (X.shape[1], 1) # 입력 차원 설정
output_units = np.unique(Y).shape[0] # 출력 유닛 수는 유니크한 레이블 수

kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
fold_no = 1

accuracies = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

    # 모델 생성 및 훈련
    model = build_advanced_model(input_shape, output_units)
    print(f'Training fold {fold_no}...')
    model.fit(X_train, Y_train, batch_size=16, epochs=20, validation_data=(X_test, Y_test))

    # 평가
    _, accuracy = model.evaluate(X_test, Y_test, verbose=0)
    accuracies.append(accuracy)
    print(f'Score for fold {fold_no}: Accuracy of {accuracy*100:.2f}%')
    fold_no += 1


Training fold 1...
Epoch 1/20
2455/3465 [====================>.........] - ETA: 24s - loss: 0.4542 - accuracy: 0.8284

KeyboardInterrupt: 

In [ ]:
# 최종 평가: 평균 정확도 및 표준 편차 계산
average_accuracy = np.mean(accuracies)
std_deviation = np.std(accuracies)
print(f'Final Mean Accuracy: {average_accuracy*100:.2f}%')
print(f'Standard Deviation of Accuracies: {std_deviation*100:.2f}%')
# 모델 요약
model.summary()